# Non-Negative MinTrace

Large collections of time series organized into structures at different aggregation levels often require their forecasts to follow their aggregation constraints and to be nonnegative, which poses the challenge of creating novel algorithms capable of coherent forecasts.

The `HierarchicalForecast` package provides a wide collection of Python implementations of hierarchical forecasting algorithms that follow nonnegative hierarchical reconciliation.

In this notebook, we will show how to use the `HierarchicalForecast` package to perform nonnegative reconciliation of forecasts on `Wiki2` dataset.

You can run these experiments using CPU or GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/NonNegativeReconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast datasetsforecast

## 1. Load Data

In this example we will use the `Wiki2` dataset. The following cell gets the time series for the different levels in the hierarchy, the summing dataframe  `S_df` which recovers the full dataset from the bottom level hierarchy and the indices of each hierarchy denoted by `tags`.

In [ ]:
import numpy as np
import pandas as pd

from datasetsforecast.hierarchical import HierarchicalData

In [ ]:
Y_df, S_df, tags = HierarchicalData.load('./data', 'Wiki2')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

In [ ]:
Y_df.head()

,unique_id,ds,y
0,Total,2016-01-01,156508
1,Total,2016-01-02,129902
2,Total,2016-01-03,138203
3,Total,2016-01-04,115017
4,Total,2016-01-05,126042


In [ ]:
S_df.iloc[:5, :5]

,de_AAC_AAG_001,de_AAC_AAG_010,de_AAC_AAG_014,de_AAC_AAG_045,de_AAC_AAG_063
Total,1,1,1,1,1
de,1,1,1,1,1
en,0,0,0,0,0
fr,0,0,0,0,0
ja,0,0,0,0,0


In [ ]:
tags

{'Views': array(['Total'], dtype=object),
 'Views/Country': array(['de', 'en', 'fr', 'ja', 'ru', 'zh'], dtype=object),
 'Views/Country/Access': array(['de_AAC', 'de_DES', 'de_MOB', 'en_AAC', 'en_DES', 'en_MOB',
        'fr_AAC', 'fr_DES', 'fr_MOB', 'ja_AAC', 'ja_DES', 'ja_MOB',
        'ru_AAC', 'ru_DES', 'ru_MOB', 'zh_AAC', 'zh_DES', 'zh_MOB'],
       dtype=object),
 'Views/Country/Access/Agent': array(['de_AAC_AAG', 'de_AAC_SPD', 'de_DES_AAG', 'de_MOB_AAG',
        'en_AAC_AAG', 'en_AAC_SPD', 'en_DES_AAG', 'en_MOB_AAG',
        'fr_AAC_AAG', 'fr_AAC_SPD', 'fr_DES_AAG', 'fr_MOB_AAG',
        'ja_AAC_AAG', 'ja_AAC_SPD', 'ja_DES_AAG', 'ja_MOB_AAG',
        'ru_AAC_AAG', 'ru_AAC_SPD', 'ru_DES_AAG', 'ru_MOB_AAG',
        'zh_AAC_AAG', 'zh_AAC_SPD', 'zh_DES_AAG', 'zh_MOB_AAG'],
       dtype=object),
 'Views/Country/Access/Agent/Topic': array(['de_AAC_AAG_001', 'de_AAC_AAG_010', 'de_AAC_AAG_014',
        'de_AAC_AAG_045', 'de_AAC_AAG_063', 'de_AAC_AAG_100',
        'de_AAC_AAG_110', 'de_AAC

We split the dataframe in train/test splits.

In [ ]:
Y_test_df = Y_df.groupby('unique_id').tail(7)
Y_train_df = Y_df.drop(Y_test_df.index)

In [ ]:
Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

## 2. Base Forecasts

The following cell computes the *base forecast* for each time series using the `ETS` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
%%capture
from statsforecast.models import ETS, Naive
from statsforecast.core import StatsForecast

In [ ]:
%%capture
fcst = StatsForecast(
    df=Y_train_df, 
    models=[ETS(season_length=7, model='ZAA'), Naive()], 
    freq='D', 
    n_jobs=-1
)
Y_hat_df = fcst.forecast(h=7)

Observe that the ETS model computes negative forecasts for some series.

In [ ]:
Y_hat_df.query('ETS < 0')

,ds,ETS,Naive
unique_id,,,
de_AAC_AAG_001,2016-12-25,-487.601532,340.0
de_AAC_AAG_001,2016-12-26,-215.634201,340.0
de_AAC_AAG_001,2016-12-27,-173.175613,340.0
de_AAC_AAG_001,2016-12-30,-290.836060,340.0
de_AAC_AAG_001,2016-12-31,-784.441040,340.0
...,...,...,...
zh_AAC_AAG_033,2016-12-31,-86.526421,37.0
zh_MOB,2016-12-26,-199.534882,1036.0
zh_MOB,2016-12-27,-69.527260,1036.0


## 3. Non-Negative Reconciliation

The following cell makes the previous forecasts coherent and nonnegative using the `HierarchicalReconciliation` class.

In [ ]:
from hierarchicalforecast.methods import MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

In [ ]:
%%capture
reconcilers = [
    MinTrace(method='ols'),
    MinTrace(method='ols', nonnegative=True)
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df,
                          S=S_df, tags=tags)

Observe that the nonnegative reconciliation method obtains nonnegative forecasts. 

In [ ]:
Y_rec_df.query('`ETS/MinTrace_method-ols_nonnegative-True` < 0')

,ds,ETS,Naive,ETS/MinTrace_method-ols,Naive/MinTrace_method-ols,ETS/MinTrace_method-ols_nonnegative-True,Naive/MinTrace_method-ols_nonnegative-True
unique_id,,,,,,,


The free reconciliation method gets negative forecasts.

In [ ]:
Y_rec_df.query('`ETS/MinTrace_method-ols` < 0')

,ds,ETS,Naive,ETS/MinTrace_method-ols,Naive/MinTrace_method-ols,ETS/MinTrace_method-ols_nonnegative-True,Naive/MinTrace_method-ols_nonnegative-True
unique_id,,,,,,,
de_DES,2016-12-25,-2553.932861,495.0,-3468.745214,495.0,2.262540e-15,495.0
de_DES,2016-12-26,-2155.228271,495.0,-2985.587125,495.0,1.356705e-30,495.0
de_DES,2016-12-27,-2720.993896,495.0,-3698.680055,495.0,6.857413e-30,495.0
de_DES,2016-12-29,-3429.432617,495.0,-2965.207609,495.0,2.456449e+02,495.0
de_DES,2016-12-30,-3963.202637,495.0,-3217.360371,495.0,3.646790e+02,495.0
...,...,...,...,...,...,...,...
zh_MOB_AAG_036,2016-12-26,75.298317,115.0,-165.799776,115.0,3.207772e-14,115.0
zh_MOB_AAG_036,2016-12-27,72.895554,115.0,-134.340626,115.0,2.308198e-14,115.0
zh_MOB_AAG_138,2016-12-25,94.796623,65.0,-47.009813,65.0,3.116938e-14,65.0


## 4. Evaluation

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
from hierarchicalforecast.evaluation import HierarchicalEvaluation

In [ ]:
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)

evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df, 
        tags=tags, benchmark='Naive'
)
evaluation.filter(like='ETS', axis=1).T

level,Overall,Views,Views/Country,Views/Country/Access,Views/Country/Access/Agent,Views/Country/Access/Agent/Topic
metric,mse-scaled,mse-scaled,mse-scaled,mse-scaled,mse-scaled,mse-scaled
ETS,1.011585,0.7358,1.190354,1.103657,1.089515,1.397139
ETS/MinTrace_method-ols,0.979163,0.698355,1.062521,1.143277,1.113349,1.354041
ETS/MinTrace_method-ols_nonnegative-True,0.945075,0.677892,1.004639,1.184719,1.141442,1.158672


Observe that the nonnegative reconciliation method performs better that its unconstrained counterpart.

### References
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Wickramasuriya, S. L., Athanasopoulos, G., & Hyndman, R. J. (2019). \"Optimal forecast reconciliation for
    hierarchical and grouped time series through trace minimization\". Journal of the American Statistical Association, 
    114 , 804–819. doi:10.1080/01621459.2018.1448825.](https://robjhyndman.com/publications/mint/).
- [Wickramasuriya, S.L., Turlach, B.A. & Hyndman, R.J. (2020). \"Optimal non-negative
    forecast reconciliation". Stat Comput 30, 1167–1182, 
    https://doi.org/10.1007/s11222-020-09930-0](https://robjhyndman.com/publications/nnmint/).